In [1]:
import urllib.request
import urllib.error
import json
import datetime
import time
import sys, os
import logging
import glob
from urllib.error import HTTPError
import configparser
from configparser import SafeConfigParser

In [2]:
def daterange( start_date, end_date ):
    if start_date <= end_date:
        for n in range( ( end_date - start_date ).days + 1 ):
            yield start_date + datetime.timedelta( n )
    else:
        for n in range( ( start_date - end_date ).days + 1 ):
            yield start_date - datetime.timedelta( n )

In [3]:
def convert(input):
    if isinstance(input, dict):
        return {convert(key): convert(value) for key, value in input.items()}
    elif isinstance(input, list):
        return [convert(element) for element in input]
    else:
        return input

In [4]:
def getJsonFileName(date, page, json_file_path):
    json_file_name = ".".join([date,str(page),'json'])
    json_file_name = "".join([json_file_path,'\\', json_file_name])
    return json_file_name

In [11]:
# get the articles from the NYTimes Article API    
def getArticles(date, query, api_key, json_file_path):
    # LOOP THROUGH THE 101 PAGES NYTIMES ALLOWS FOR THAT DATE
    for page in range(2):
        for n in range(1): # 5 tries
            try:
                request_string = "http://api.nytimes.com/svc/search/v2/articlesearch.json?q="+query+"&begin_date=" + date + "&end_date=" + date + "&page=" + str(page) + "&api-key=" + api_key
                print(request_string)
                response = urllib.request.urlopen(request_string)
                content = response.read().decode('utf-8')
                #print(content)
                if content:
                   
                    articles = convert(json.loads(content))
                    #print(articles)
                    # if there are articles here
                    if len(articles["response"]["docs"]) >= 1:
                        docs = articles["response"]["docs"]
                        #print("entering")
                        #print(docs)
                        json_file_name = getJsonFileName(date, page, json_file_path)
                        json_file = open(json_file_name, 'w')
                        json_file.write(content)
                        json_file.close()
                    # if no more articles, go to next date
                    else:
                        return
                time.sleep(3) # wait so we don't overwhelm the API
            except HTTPError as e:
                logging.error("HTTPError on page %s on %s (err no. %s: %s) Here's the URL of the call: %s", page, date, e.code, e.reason, request_string)
                if e.code == 403:
                    print("Script hit a snag and got an HTTPError 403. Check your log file for more info.")
                    return
                if e.code == 429:
                    print ("Waiting. You've probably reached an API limit.")
                    time.sleep(30) # wait 30 seconds and try again
            except: 
                logging.error("Error on %s page %s: %s", date, file_number, sys.exc_info()[0])
                continue

In [ ]:
json_folder = glob.glob(r'D:\SEM 2\Python\Assignments\chintha_priyal_spring2017\midterm\nyt\janjson')
for file in json_folder:
    with open(file) as json_file:
        json_data = json.load(json_file)
        #print(json_data)
        #if 'docs' not in json_data.keys():
            #print('yes')
        response = json_data["response"]
        docs = response['docs']
        #print(len(docs))
        for dicts in docs:
            section_name = dicts["section_name"]
            
            #print(section_name)


In [ ]:
def main():
    
    config = SafeConfigParser()
    script_dir = os.getcwd()
    config_file = os.path.join(script_dir, 'config\config.cfg')
    config.read(config_file)
    
    json_file_path = config.get('files','json_folder')
    #tsv_file_name = config.get('files','tsv_file')
    log_file = config.get('files','logfile')
    
    api_key = config.get('nytimes','api_key')    
    start = datetime.date( year = int(config.get('nytimes','start_year')), month = int(config.get('nytimes','start_month')), day = int(config.get('nytimes','start_day')) )
    end = datetime.date( year = int(config.get('nytimes','end_year')), month = int(config.get('nytimes','end_month')), day = int(config.get('nytimes','end_day')) )
    query = config.get('nytimes','query')
        
    logging.basicConfig(filename=log_file, level=logging.INFO)
    
    logging.info("Getting started.") 
    try:
        # LOOP THROUGH THE SPECIFIED DATES
        for date in daterange( start, end ):
            date = date.strftime("%Y%m%d")
            logging.info("Working on %s." % date)
            getArticles(date, query, api_key, json_file_path)
            #parseArticles(date, tsv_file_name, json_file_path)
    except:
        logging.error("Unexpected error: %s", str(sys.exc_info()[0]))
    finally:
        logging.info("Finished.")

if __name__ == '__main__' :
    main()

C:\Users\chint\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: DeprecationWarning: The SafeConfigParser class has been renamed to ConfigParser in Python 3.2. This alias will be removed in future versions. Use ConfigParser directly instead.
  app.launch_new_instance()


http://api.nytimes.com/svc/search/v2/articlesearch.json?q=Crime+US&begin_date=20160801&end_date=20160801&page=0&api-key=9861caf75f764f97a4e1c073d5036684
http://api.nytimes.com/svc/search/v2/articlesearch.json?q=Crime+US&begin_date=20160801&end_date=20160801&page=1&api-key=9861caf75f764f97a4e1c073d5036684
http://api.nytimes.com/svc/search/v2/articlesearch.json?q=Crime+US&begin_date=20160802&end_date=20160802&page=0&api-key=9861caf75f764f97a4e1c073d5036684
http://api.nytimes.com/svc/search/v2/articlesearch.json?q=Crime+US&begin_date=20160802&end_date=20160802&page=1&api-key=9861caf75f764f97a4e1c073d5036684
http://api.nytimes.com/svc/search/v2/articlesearch.json?q=Crime+US&begin_date=20160803&end_date=20160803&page=0&api-key=9861caf75f764f97a4e1c073d5036684
http://api.nytimes.com/svc/search/v2/articlesearch.json?q=Crime+US&begin_date=20160803&end_date=20160803&page=1&api-key=9861caf75f764f97a4e1c073d5036684
http://api.nytimes.com/svc/search/v2/articlesearch.json?q=Crime+US&begin_date=2016